In [1]:
using DrWatson;
@quickactivate "NumericalAnalysis"

In [2]:
using LinearAlgebra;

# Triangular Systems: Backward and Forward Substitution

When going through this notebook, it is beneficial to watch the corresponding video lecture on [triangular linear systems](https://www.youtube.com/watch?v=w78GX22j-8g&list=PLvUvOH0OYx3BcZivtXMIwP6hKoYv0YvGn&index=6).

A linear system with a diagonal coefficient matrix is trivial to solve since the equations become uncoupled. The next simplest linear systems to solve are **triangular systems**. Let's start with an example. Consider the following linear system
$$
\begin{align*}
-2x + y - 3z &= 1 \\
      4y + 2z&= -3 \\
           -4z&=8
\end{align*}
$$
We can immediately solve for $z$ using the third equation since $z = \frac{8}{-4}=-2$. From the second equation then
$$
y = (-3 - 2y)/4 = \frac{1}{4}.
$$
Finally, we can use the first equation to solve for $x$ as follows.
$$
x = (1 - y + 3z)/-2 = \frac{21}{8}
$$
This process where we start with the final equation and then work our way to the first equation solving for one additional variable at each step is called backward substition. Let's use Julia to check our solution. 

In [3]:
x = 21/8;
y = 1/4;
z = -2;
println(-2*x + y -3*z)
println(4*y + 2*z)
println(-4*z)

1.0
-3.0
8


Notice that in matrix/vector notation this linear system becomes

$$
\begin{align*}
\left[\begin{array}{ccc} -2 & 1 & -3 \\ 0 & 4 & 2 \\ 0 & 0 & -4 \end{array}\right] \left[\begin{array}{c} x \\ y \\ z \end{array} \right] = \left[\begin{array}{c} 1 \\ -3 \\ 8 \end{array} \right].
\end{align*}
$$

Since all of the entries in the coefficient matrix that lie below the main diagonal are zero, we say that this matrix is upper triangular. As our example illustrates, we can solve upper triangular systems algorithmically using backward substitution. You should note that at each step in bacward substitution we are dividing by a diagonal entry of the matrix. This is a problem if a diagonal entry is equal to zero. However, if a trianglar matrix has a zero diagonal entry then the matrix is singular. You will justify this observation in the homework.   

A linear system with a coefficient matrix $U$ that satisfies that all entries **below** the main diagonal are equal to zero is called an **upper triangular** system. An upper triangular system has the following form: 

$$
\left[\begin{array}{cccc} u_{11} & u_{12} & \cdots & u_{1n}\\ 0 & u_{22} & \cdots & u_{2n} \\ \vdots & \cdots & \ddots & \vdots \\ 0 & 0 & \cdots & u_{nn} \end{array}\right] \left[\begin{array}{c} x_{1} \\ x_{2} \\ \vdots \\ x_{n} \end{array} \right] = \left[\begin{array}{c} b_{1} \\ b_{2} \\ \vdots \\ b_{n} \end{array} \right].
$$

We say that a matrix $U$ is upper triangular if it has the form

$$
U = \left[\begin{array}{cccc} u_{11} & u_{12} & \cdots & u_{1n}\\ 0 & u_{22} & \cdots & u_{2n} \\ \vdots & \cdots & \ddots & \vdots \\ 0 & 0 & \cdots & u_{nn} \end{array}\right]. 
$$

Let's derive our backward substitution algorithm (sometimes called back solve) in general. Clearly we have that
$$
x_{n} = \frac{b_{n}}{u_{nn}}.
$$
Then for $i=n-1,n-2,\ldots,2,1$ (counting downwards) we can use the $i$-th row to solve for $x_{i}$ after having solve for all of the previous $n-i$ unknowns. That is, we have
$$
x_{i} = (b_{i} - u_{i,i+1}x_{i+1} - u_{i,i+2}x_{i+2} - \cdots -u_{i,n}x_{n})/u_{i,i}.
$$
Let's make an observation that will allow us to simplify our implementation of this algorithm. We have that 
$$
\begin{align*}
x_{i} &= (b_{i} - u_{i,i+1}x_{i+1} - u_{i,i+2}x_{i+2} - \cdots -u_{i,n}x_{n})/u_{i,i} \\
&= (b_{i} - (u_{i,i+1}x_{i+1} + u_{i,i+2}x_{i+2} + \cdots + u_{i,n}x_{n}))/u_{i,i},
\end{align*}
$$
and the expression $u_{i,i+1}x_{i+1} + u_{i,i+2}x_{i+2} + \cdots + u_{i,n}x_{n}$ looks very similar to a dot product. In fact, it is the dot product of the last $n-i$ entries of row $i$ of $U$ with the last $n-i$ entries of the vector $x$. 

A linear system with a coefficient matrix $L$ that satisfies that all entries **above** the main diagonal are equal to zero is called a **lower triangular** system. A lower triangular system has the following form: 

$$
\left[\begin{array}{cccc} l_{11} & 0 & \cdots & 0\\ l_{21} & l_{22} & \cdots & 0 \\ \vdots & \cdots & \ddots & \vdots \\ l_{n1} & l_{n2} & \cdots & l_{nn} \end{array}\right] \left[\begin{array}{c} x_{1} \\ x_{2} \\ \vdots \\ x_{n} \end{array} \right] = \left[\begin{array}{c} b_{1} \\ b_{2} \\ \vdots \\ b_{n} \end{array} \right].
$$

We say that a matrix $L$ is lower triangular if it has the form

$$
L = \left[\begin{array}{cccc} l_{11} & 0 & \cdots & 0\\ l_{21} & l_{22} & \cdots & 0 \\ \vdots & \cdots & \ddots & \vdots \\ l_{n1} & l_{n2} & \cdots & l_{nn} \end{array}\right].
$$

In the homework you will derive an algorithm called forward substitution or forward solve similar to back substitution  that can be used to efficiently solve lower triangular systems. Now, let's look at the Julia implementation of these two algorithms. 

In [4]:
"""
    forwardsub(L,b)

Implements the forward substition algorithm to solve the linear system \$Ly=b\$, 
where \$L\$ is an \$n \\times n\$ lower triangular matrix and \$b\$ is a vector of length \$n\$.

# Example
```julia-repl
julia> L = [1. 0. 0.;2. -1. 0.; 1. -1. 1.];
julia> b = ones(3);
julia> y = forwardsub(L,b)
```

"""
function forwardsub(L,b)
    n = size(L)[1]; # number of rows
    y = zeros(n); # initialize solution vector
    y[1] = b[1]/L[1,1];
    for i=2:n
        y[i] = (b[i] - dot(L[i, 1:i-1],y[1:i-1])) / L[i,i];
    end
    return y
end

forwardsub

In [5]:
"""
    backsub(L,b)

Implements the back substition algorithm to solve the linear system \$Ux=y\$, 
where \$U\$ is an \$n \\times n\$ upper triangular matrix and \$y\$ is a vector of length \$n\$.

# Example
```julia-repl
julia> U = [-1.0 2.0 1.0;0. 3.0 -2.0;0.0 0.0 -1.0];
julia> y = ones(3);
julia> x = backsub(U,y)
```

"""
function backsub(U,y)
    n = size(U)[1]; # number of rows
    x = zeros(n); # initialize solution vector
    x[n] = y[n]/U[n,n];
    for i=n-1:-1:1
        x[i] = (y[i] - dot(U[i, i+1:n],x[i+1:n])) / U[i,i];
    end
    return x
end

backsub

Let's illustrate the use of our implementations of backward and forward substitution.

In [6]:
L = tril(rand(5,5)) # a random 5 by 5 lower triangular system 

5×5 Matrix{Float64}:
 0.307981    0.0       0.0       0.0       0.0
 0.685584    0.120177  0.0       0.0       0.0
 0.00553337  0.556633  0.456395  0.0       0.0
 0.594922    0.617126  0.242596  0.621268  0.0
 0.423625    0.749804  0.303646  0.783503  0.708762

In [7]:
U = triu(rand(5,5)) # a random 5 by s upper triangular system

5×5 Matrix{Float64}:
 0.994386  0.799054  0.24876   0.210439  0.292834
 0.0       0.395887  0.823768  0.190783  0.59801
 0.0       0.0       0.24008   0.174776  0.176398
 0.0       0.0       0.0       0.328939  0.265587
 0.0       0.0       0.0       0.0       0.356201

In [8]:
b = ones(5) # a right hand side vector of length 5 with each entry equal to 1

5-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0

In [9]:
y = forwardsub(L,b) # use forward substitution 

5-element Vector{Float64}:
   3.246951267605246
 -10.202139199265977
  14.5945571721438
   2.9355167486152736
   0.7654848032074343

In [10]:
x = backsub(U,y) # use backward substitution 

5-element Vector{Float64}:
  103.96324986030268
 -144.79920145112374
   53.97781526348358
    7.189057737980728
    2.149023138426974

In [11]:
b - L*y # compute the residual 

5-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [12]:
y - U*x # compute the residual 

5-element Vector{Float64}:
 8.881784197001252e-16
 0.0
 0.0
 0.0
 0.0

Recall that we previously showed that to solve the system $LUx=b$. Now can split the problem $LUx = b$ into two subsystems:

1. $Ly=b$ which has solution $y=L^{-1}b$ that is computed using forward substitution, and
  
2. $Ux=y$ which has solution $x=U^{-1}y=U^{-1}L^{-1}b$ that is computed by backward substitution.

We can illustrate this computationally. For example, 

In [13]:
b - L*U*x

5-element Vector{Float64}:
 -4.884981308350689e-15
 -4.440892098500626e-15
  1.9984014443252818e-15
  1.7763568394002505e-15
 -3.552713678800501e-15

# Looking Forward to LU Factorization

Consider a general square linear system $Ax=b$ with $A$ an $n\times n$ matrix. In the next lecture we will show that it is often possible to factorize $A$ as $A=LU$ with $L$ lower triangular and $U$ upper triangular. As we just pointed out, this provides a method for solving $Ax=b$ since we can use the factorization $A=LU$, that is, the so-called **LU factorization** of $A$ to solve the linear system by splitting $Ax=LUx = b$ into two subsystems. Furthmore, we will see that $LU$ factorization arises from, and in fact is equivalent to, Gaussian elimination. In preparation for the next lecture, it is suggested that you watch the lecture video on [LU factorization](https://www.youtube.com/watch?v=aFbjNVZNYYk&list=PLvUvOH0OYx3BcZivtXMIwP6hKoYv0YvGn&index=8). 